In [1]:
import numpy as np

In [2]:
def E(i, j, t, Qx, a, b):
    """
    Recursive definition of Hermite Gaussian coefficients.

    Returns a float.
    a: orbital exponent on Gaussian 'a' (e.g. alpha in the text)
    b: orbital exponent on Gaussian 'b' (e.g. beta in the text)
    i,j: orbital angular momentum number on Gaussian 'a' and 'b'
    t: number nodes in Hermite (depends on type of integral,
        e.g. always zero for overlap integrals)
    Qx: distance between origins of Gaussian 'a' and 'b'
    """
    p = a + b
    q = a * b / p
    if (t < 0) or (t > (i + j)):
        # out of bounds for t
        return 0.0
    elif i == j == t == 0:
        # base case
        return np.exp(-q * Qx * Qx)  # K_AB
    elif j == 0:
        # decrement index i
        return (
            (1 / (2 * p)) * E(i - 1, j, t - 1, Qx, a, b)
            - (q * Qx / a) * E(i - 1, j, t, Qx, a, b)
            + (t + 1) * E(i - 1, j, t + 1, Qx, a, b)
        )
    else:
        # decrement index j
        return (
            (1 / (2 * p)) * E(i, j - 1, t - 1, Qx, a, b)
            + (q * Qx / b) * E(i, j - 1, t, Qx, a, b)
            + (t + 1) * E(i, j - 1, t + 1, Qx, a, b)
        )

In [3]:
def overlap(a, lmn1, A, b, lmn2, B):
    """
    Evaluates overlap integral between two Gaussians

    Returns a float.
    a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
    b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
    lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
          for Gaussian 'a'
    lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
    A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
    B:    list containing origin of Gaussian 'b'
    """
    l1, m1, n1 = lmn1  # shell angular momentum on Gaussian 'a'
    l2, m2, n2 = lmn2  # shell angular momentum on Gaussian 'b'
    S1 = E(l1, l2, 0, A[0] - B[0], a, b)  # X
    S2 = E(m1, m2, 0, A[1] - B[1], a, b)  # Y
    S3 = E(n1, n2, 0, A[2] - B[2], a, b)  # Z

    return S1 * S2 * S3 * np.power(np.pi / (a + b), 1.5)

In [ ]:
def kinetic(a, lmn1, A, b, lmn2, B):
    """
    Evaluates kinetic energy integral between two Gaussians

    Returns a float.
    a: orbital exponent on Gaussian 'a' (e.g. alpha in the text)
    b: orbital exponent on Gaussian 'b' (e.g. beta in the text)
    lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
          for Gaussian 'a'
    lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
    A: list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
    B: list containing origin of Gaussian 'b'
    """
    l1, m1, n1 = lmn1
    l2, m2, n2 = lmn2
    term0 = (
        b * (2 * (l2 + m2 + n2) + 3) * overlap(a, (l1, m1, n1), A, b, (l2, m2, n2), B)
    )
    term1 = (
        -2
        * np.power(b, 2)
        * (
            overlap(a, (l1, m1, n1), A, b, (l2 + 2, m2, n2), B)
            + overlap(a, (l1, m1, n1), A, b, (l2, m2 + 2, n2), B)
            + overlap(a, (l1, m1, n1), A, b, (l2, m2, n2 + 2), B)
        )
    )
    term2 = -0.5 * (
        l2 * (l2 - 1) * overlap(a, (l1, m1, n1), A, b, (l2 - 2, m2, n2), B)
        + m2 * (m2 - 1) * overlap(a, (l1, m1, n1), A, b, (l2, m2 - 2, n2), B)
        + n2 * (n2 - 1) * overlap(a, (l1, m1, n1), A, b, (l2, m2, n2 - 2), B)
    )
    return term0 + term1 + term2

In [ ]:
def T(a, b):
    """
    Evaluates kinetic energy between two contracted Gaussians

    Returns float.
    Arguments:
    a: contracted Gaussian 'a', BasisFunction object
    b: contracted Gaussian 'b', BasisFunction object
    """
    t = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            t += (
                a.norm[ia]
                * b.norm[ib]
                * ca
                * cb
                * kinetic(a.exps[ia], a.shell, a.origin, b.exps[ib], b.shell, b.origin)
            )
    return t